In [1]:
import pandas as pd
import numpy as np

# Load a CSV matrix (ignore first column)
def import_matrix(file_path: str) -> np.ndarray:
    try:
        data = pd.read_csv(file_path, header=None)
        return np.nan_to_num(data.iloc[:, 1:].to_numpy(), nan=0.0)
    except FileNotFoundError:
        raise FileNotFoundError(f"File {file_path} not found")

# Load parameter placeholders
try:
    weight_raw = import_matrix("W.csv")
    bias_raw = import_matrix("b.csv")
    print("Weight matrix shape:", weight_raw.shape)
    print("Bias matrix shape:", bias_raw.shape)
except Exception as e:
    print(f"Error loading matrices: {e}")
    exit(1)


# Partition weights and biases per layer

def slice_weights(full_w, arch):
    mats = []
    start = 0
    for in_size, out_size in arch:
        mats.append(full_w[start:start + in_size, :out_size])
        start += in_size
    return mats

def slice_biases(full_b, outs):
    return [full_b[i, :dim] for i, dim in enumerate(outs)]

# Model architecture
net_arch = [(6, 64), (64, 32), (32, 16), (16, 1)]
out_shapes = [64, 32, 16, 1]

# Validate shapes
expected_w_rows = sum(in_size for in_size, _ in net_arch)
if weight_raw.shape[0] != expected_w_rows or weight_raw.shape[1] != max(out_shapes):
    raise ValueError("Weight matrix shape mismatch")
if bias_raw.shape[0] != len(out_shapes) or bias_raw.shape[1] != max(out_shapes):
    raise ValueError("Bias matrix shape mismatch")

weight_layers = slice_weights(weight_raw, net_arch)
bias_layers = slice_biases(bias_raw, out_shapes)
W1, W2, W3, W4 = weight_layers
b1, b2, b3, b4 = bias_layers

# Activation functions
def ReLU(Z):
    return np.maximum(0, Z)

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def Gradient_ReLU(Z):
    return (Z > 0).astype(float)

# Forward propagation
def forward_propagation(X, W1, b1, W2, b2, W3, b3, W4, b4):
    Z1 = X @ W1 + b1
    A1 = ReLU(Z1)
    
    Z2 = A1 @ W2 + b2
    A2 = ReLU(Z2)
    
    Z3 = A2 @ W3 + b3
    A3 = ReLU(Z3)
    
    Z4 = A3 @ W4 + b4
    A4 = sigmoid(Z4)
    
    return Z1, A1, Z2, A2, Z3, A3, Z4, A4

# Backward propagation
def backward_propagation(W1, b1, W2, b2, W3, b3, W4, b4, Z1, A1, Z2, A2, Z3, A3, Z4, A4, X, Y):
    m = X.shape[0]
    dZ4 = A4 - Y
    dW4 = (A3.T @ dZ4) / m
    db4 = np.sum(dZ4, axis=0, keepdims=True) / m
    
    dZ3 = (dZ4 @ W4.T) * Gradient_ReLU(Z3)
    dW3 = (A2.T @ dZ3) / m
    db3 = np.sum(dZ3, axis=0, keepdims=True) / m
    
    dZ2 = (dZ3 @ W3.T) * Gradient_ReLU(Z2)
    dW2 = (A1.T @ dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m
    
    dZ1 = (dZ2 @ W2.T) * Gradient_ReLU(Z1)
    dW1 = (X.T @ dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m
    
    return dW1, db1, dW2, db2, dW3, db3, dW4, db4

# Update parameters
def update_parameters(W1, b1, W2, b2, W3, b3, W4, b4, dW1, db1, dW2, db2, dW3, db3, dW4, db4, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W3 -= learning_rate * dW3
    b3 -= learning_rate * db3
    W4 -= learning_rate * dW4
    b4 -= learning_rate * db4
    
    return W1, b1, W2, b2, W3, b3, W4, b4

# Get predictions
def get_predictions(A4):
    return (A4 > 0.5).astype(int)

def get_accuracy(predictions, Y):
    return np.mean(predictions == Y)

# Gradient descent
def gradient_descent(X, Y, alpha, iterations):
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3, Z4, A4 = forward_propagation(X, W1, b1, W2, b2, W3, b3, W4, b4)
        dW1, db1, dW2, db2, dW3, db3, dW4, db4 = backward_propagation(W1, b1, W2, b2, W3, b3, W4, b4, 
                                                                     Z1, A1, Z2, A2, Z3, A3, Z4, A4, X, Y)
        W1, b1, W2, b2, W3, b3, W4, b4 = update_parameters(W1, b1, W2, b2, W3, b3, W4, b4, 
                                                          dW1, db1, dW2, db2, dW3, db3, dW4, db4, alpha)
        
        if i % 100 == 0:
            predictions = get_predictions(A4)
            accuracy = get_accuracy(predictions, Y)
            print(f"Iteration {i}, Accuracy: {accuracy}")
    
    return W1, b1, W2, b2, W3, b3, W4, b4

# Export gradients function
def export_grads(grad_dict, arch, out_shapes, W_template, B_template):
    dW_full = np.full_like(W_template, np.nan, dtype=np.float32)
    dB_full = np.full_like(B_template, np.nan, dtype=np.float32)

    # Reuse slicing logic for weights
    start = 0
    for i, (in_size, out_size) in enumerate(arch):
        dW_full[start:start + in_size, :out_size] = grad_dict[f"dW{i+1}"]
        start += in_size

    # Reuse slicing logic for biases
    for i, dim in enumerate(out_shapes):
        dB_full[i, :dim] = grad_dict[f"db{i+1}"]

    try:
        pd.DataFrame(dW_full).to_csv("dw_123456.csv", index=False, header=False, float_format="%.16f")
        pd.DataFrame(dB_full).to_csv("db_123456.csv", index=False, header=False, float_format="%.16f")
    except Exception as e:
        raise IOError(f"Error saving gradients: {e}")

# Example execution
student_code = "123456"
try:
    x_in = np.array([int(ch) for ch in student_code], dtype=float).reshape(1, -1) / 9.0
    if x_in.shape[1] != net_arch[0][0]:
        raise ValueError(f"Input size {x_in.shape[1]} does not match expected {net_arch[0][0]}")
    y_target = np.array([[0]])

    # Forward pass
    Z1, A1, Z2, A2, Z3, A3, Z4, A4 = forward_propagation(x_in, W1, b1, W2, b2, W3, b3, W4, b4)
    
    # Backward pass
    dW1, db1, dW2, db2, dW3, db3, dW4, db4 = backward_propagation(W1, b1, W2, b2, W3, b3, W4, b4, 
                                                                 Z1, A1, Z2, A2, Z3, A3, Z4, A4, x_in, y_target)

    # Bundle gradients for export
    grad_bundle = {
        "dW1": dW1, "db1": db1,
        "dW2": dW2, "db2": db2,
        "dW3": dW3, "db3": db3,
        "dW4": dW4, "db4": db4,
    }

    export_grads(grad_bundle, net_arch, out_shapes, weight_raw, bias_raw)
    
except Exception as e:
    print(f"Error during execution: {e}")
    exit(1)

Weight matrix shape: (118, 64)
Bias matrix shape: (4, 64)


In [2]:
# Evaluation Script for student
import os
import numpy as np
import csv

ID = '123456'  # Index number

file_name = ['dw_123456.csv', 'db_123456.csv']  # generated files
truth_path = ['true_dw_123456.csv', 'true_db_123456.csv']  # true files
threshold = 0.05


def read_file(name):
    data = []
    with open(name) as f:
        reader = csv.reader(f)
        for row in reader:
            cleaned_row = [np.float32(x) if x != '' else np.float32(0) for x in row]
            data.append(cleaned_row)
    return data


def grade(l0, l1, th):
    dif = np.mean(np.abs(l0 - l1).astype(float) / (0.1 + l1))
    return 1 if dif <= th else 0


def compare(sub, true, threshold=0):
    if len(sub) != len(true):
        return 0
    scores = []
    for row_sub, row_true in zip(sub, true):
        l0 = np.array(row_sub).astype(float)
        l1 = np.array(row_true).astype(float)
        if len(l0) != len(l1):
            return 0
        scores.append(grade(l0, l1, threshold))
    return scores


# Read all files
student_grads = [read_file(f) for f in file_name]
true_grads = [read_file(f) for f in truth_path]

# Compare and score
score = []
for sub, true in zip(student_grads, true_grads):
    s = compare(sub, true, threshold)
    if isinstance(s, list):
        score += s
    else:
        score.append(s)

print("Total score:",np.sum(score))

Total score: 122
